In [7]:
import re, json

In [8]:
with open ('sripts.json','r',encoding='utf-8') as inp:
    scripts = json.load(inp)

Определим регулярные выражения для того, чтобы убрать html-разметку и пояснения типа \[door screeches\] из текста (оставляем только реплики)

In [9]:
re_tag = re.compile(r'<.*?>')
re_sound = re.compile(r'\[.*?\]')

In [10]:
full_text = ''
for script in scripts:
    full_text += script

В качестве токенов будет использовать словоформы (в том числе, содержащие дефис или заканчивающиеся на апостроф), знаки препинания и числа:

In [11]:
re_token = re.compile(u"[a-zA-Z0-9-]+'?|[.,:;?!]+")

In [12]:
full_text = re_tag.sub('',full_text)
full_text = re_sound.sub('',full_text)

In [13]:
tokens = re_token.findall(full_text)

Вычленим все триграммы:

In [14]:
trigrams = []
for i in range(0,len(tokens)-3):
    trigrams.append(tokens[i]+' '+tokens[i+1]+' '+tokens[i+2])

Найдём частоты триграмм (чтобы частоты нашлись быстрее, не будем создавать словарь, а воспользуемся статистическим модулем pandas, написанным изначально на C и поставляющимся в скомпилированном виде, что увеличивает скорость работы программ)

In [15]:
import pandas

In [16]:
trigram_frequencies = pandas.Series(trigrams).value_counts().to_dict()

Теперь разделим триграммы на токены и создадим датафрейм типа token1-token2-token3-frequency:

In [17]:
trigram_freq_frame = []
for i in trigram_frequencies:
    trigram_freq_frame.append(i.split(' ') + [trigram_frequencies[i]])

In [18]:
trigram_freq_frame = pandas.DataFrame(trigram_freq_frame, columns = ['word1','word2','word3','frequency'])

In [19]:
trigram_freq_frame.sort_values(by='frequency',ascending=False).head()

,word1,word2,word3,frequency
0,",",sir,.,1403
1,",",Mr,.,1096
2,Mr,.,Spock,1063
3,",",captain,.,812
4,.,Mr,.,617


Сохраним результат в csv

In [20]:
trigram_freq_frame.to_csv('trigram_frequencies.csv')

Поиграемся с Марковской цепью:

In [21]:
import pandas
trigram_freq_frame = pandas.read_csv('trigram_frequencies.csv')

In [22]:
import random

In [23]:
n = len(trigram_freq_frame.index)

Сначала выберем рандомную триграмму, потом по правилу Марковской цепи будем присоединять к ней токены (ограничимся длиной 28+2 = 30)

In [67]:
#random_index = random.randint(0,n)
random_index = random.choice(trigram_freq_frame.loc[trigram_freq_frame['word1'].str.istitle()].index)
trm = trigram_freq_frame.loc[random_index]
s = trm['word1']+' '+trm['word2']+' '+trm['word3']
for i in range(28):
    most_prob_index = trigram_freq_frame.loc[(trigram_freq_frame['word1']==trm['word2'])&(trigram_freq_frame['word2']==trm['word3'])]['frequency'].idxmax()
    new_trm = trigram_freq_frame.loc[most_prob_index]
    s += ' ' + new_trm['word3']
    trm = new_trm
s

"Damage Control , this is the captain . - I don' t know . I am not Janice Lester . I am not Janice Lester . I am not Janice Lester"

In [44]:
random.choice(trigram_freq_frame.index)

210762

In [54]:
help(str)

Help on class str in module builtins:

class str(object)
 |  str(object='') -> str
 |  str(bytes_or_buffer[, encoding[, errors]]) -> str
 |  
 |  Create a new string object from the given object. If encoding or
 |  errors is specified, then the object must expose a data buffer
 |  that will be decoded using the given encoding and error handler.
 |  Otherwise, returns the result of object.__str__() (if defined)
 |  or repr(object).
 |  encoding defaults to sys.getdefaultencoding().
 |  errors defaults to 'strict'.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __format__(...)
 |      S.__format__(format_spec) -> str
 |      
 |      Return a formatted version of S as described by format_spec.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getatt

Теперь научимся отлавливать триграммы, которых нет в датафрейме:

In [70]:
non_match = trigram_freq_frame.loc[trigram_freq_frame['word1']=='fuck']

In [98]:
non_match.values.size

0

In [74]:
if non_match['word1'] == []:
    print('ok')

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [82]:
if 'Mr' in trigram_freq_frame['word1'].values:
    print('ok')

ok


In [84]:
if 'fuck' in trigram_freq_frame['word1'].values:
    print('not ok')

In [99]:
trigram_freq_frame.loc[trigram_freq_frame['word1']=='fuck'].idxmax()

ValueError: attempt to get argmax of an empty sequence